In [1]:
import requests
import json
import pandas as pd
import time
import random

In [2]:
from datetime import datetime, timedelta

# Current time in UTC
now = datetime.utcnow()

# Calculate the start time and end time
start_time = now.replace(minute=0, second=0, microsecond=0)
start_time = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
end_time = (now + timedelta(weeks=1)).strftime("%Y-%m-%dT%H:%M:%SZ")  # 2 weeks later
end_time = (now + timedelta(weeks=2)).replace(
        hour=23,
        minute=59,
        second=59,
        microsecond=0
    )
end_time = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL
print(start_time)
print(end_time)

2025-01-28T19:00:00Z
2025-02-11T23:59:59Z


In [3]:
type(end_time)

str

In [4]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store each batch of matches data
all_matches = []

# Loop over drillDownTagIds ranges in steps of 900
for start in range(200, 14000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        f"startTimeFrom={start_time}"
        f"startTimeTo={end_time}&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        for event in data.get('data', {}).get('events', []):
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
            
            matches.append(match)

        # Add the batch of matches to the all_matches list
        all_matches.extend(matches)

        # # Sleep for a random duration between 1 and 4 seconds, with variability of 0.1 ms
        # time.sleep(random.uniform(0, 1.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
print(final_matches_df)

An error occurred for drilldown ID range 800–1000: 'NoneType' object is not iterable
     event_id                                         match_name  \
0     6667895                        Wrexham AFC vs Stevenage FC   
1     6667819               Bolton Wanderers vs Northampton Town   
2     6667889             Stockport County FC vs Shrewsbury Town   
3     6667822                Charlton Athletic vs Bristol Rovers   
4     6667866                     Mansfield Town vs Crawley Town   
...       ...                                                ...   
2270  6642895                       Justin Gaethje vs Dan Hooker   
2271  6686874         Piter, K/Stollar, F vs Grabher J / Kraus S   
2272  6642836                          Rob Font vs Dominick Cruz   
2273  6686876  Siniakova K / Zhang S vs Gleason Q / Zimmermann K   
2274  6686875       Sramkova R / Tomova V vs Appleton E / Tang Q   

                start_time              home_team                 away_team  \
0     2025-01-28T19

In [4]:
len(final_matches_df)

2218

In [5]:
final_matches_df['event_id'].nunique()

1418

In [6]:
final_matches_df[final_matches_df['match_name'].str.contains('PSV')]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
505,6351955,NEC vs PSV,2025-02-01T20:00:00Z,NEC,PSV,Nederland Eredivisie,Nederland,Voetbal,7.00,1.36
513,6626894,Jong PSV vs VVV Venlo,2025-01-27T19:00:00Z,Jong PSV,VVV Venlo,Nederland Eerste Divisie,Nederland,Voetbal,2.40,2.65
520,6682208,PSV Eindhoven vs FC Twente,2025-02-01T15:30:00Z,PSV Eindhoven,FC Twente,Nederland Eredivisie Vrouwen,Nederland,Voetbal,1.36,5.75
1153,6248696,PSV vs Liverpool,2025-01-29T20:00:00Z,PSV,Liverpool,UEFA Champions League,Europees,Voetbal,2.80,2.37


In [7]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        # Define the URL for the current batch
        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&"
            "includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"
        )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
115813  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115814  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115815  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115816  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115817  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [8]:
final_df[final_df['Event Name'].str.contains('PSV')]['Event Name'].unique()

array(['NEC vs PSV', 'Jong PSV vs VVV Venlo',
       'PSV Eindhoven vs FC Twente', 'PSV vs Liverpool'], dtype=object)

In [9]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = final_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()
# Remove columns that end with '_x'
final_df = final_df[[col for col in final_df.columns if not col.endswith('_x')]]

# Rename columns to remove '_y'
final_df.columns = final_df.columns.str.replace('_y', '', regex=False)

In [10]:
# Replace 'A' with 2 and 'H' with 1 in the outcome_subtype column
final_df['Outcome SubType'] = final_df['Outcome SubType'].replace({'A': '2', 'H': '1'})

# Display the updated final_df
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
175067  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175068  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175069  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175070  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175071  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [11]:
set(final_df[final_df['sport'].str.contains('Voetbal')].competition)

{'Andorra First Division',
 'Argentinië Superliga',
 'Australië A-League',
 'Australië W-League',
 'Azerbeidzjan Premier League',
 'België 1B Pro League',
 'België Jupiler Pro League',
 'Brazil Campeonato Paranaense',
 'Brazil Gaucho A1',
 'Brazil Supercup',
 'Brazilië Cearense',
 'Brazilië Mineiro Modulo I',
 'Brazilië Paulista A1\n',
 'Colombia Categoría Primera A',
 'Costa Rica Ascenso League',
 'Costa Rica Primera Division',
 'Cyprus Beker',
 'Cyprus First Division',
 'Duitsland 2.Bundesliga',
 'Duitsland 3.Liga',
 'Duitsland Bundesliga',
 'Egypte Premier League',
 'Egypte Second Division',
 'El Salvador Primera Division',
 'Engeland Championship',
 'Engeland FA Super League Vrouwen',
 'Engeland League 1',
 'Engeland League 2',
 'Engeland National League',
 'Engeland National League Noord',
 'Engeland National League Zuid',
 'Engeland Premier League',
 'Ethiopië Premier League',
 'Europa Super League',
 'Europees Kampioenschap',
 'Frankrijk Ligue 1',
 'Frankrijk Ligue 2',
 'Frankri

In [12]:
final_df[(final_df['Event Name'].str.contains('Milan')) & final_df['sport']=='Voetbal']

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition


In [13]:
[i for i in list(set(final_df['Market Name'])) if 'No Bet' in i]

['1e Helft: Draw No Bet',
 '3e Periode - Draw No Bet',
 '2e Helft: Draw No Bet',
 '2e Periode - Draw No Bet',
 'Draw No Bet',
 '1e Periode - Draw No Bet']

In [14]:
final_output_name = 'totoAllSports' + start_time + '.csv'
final_df.to_csv(final_output_name)

## Check id ranges

In [15]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store all match data
all_matches = []

# Initialize a dictionary to store sports and competitions by drillDownTagIds range
range_summary = {}

# Loop over drillDownTagIds ranges in steps of 1000
for start in range(0, 22000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2025-01-15T23:00:00Z&"
        "startTimeTo=2025-01-30T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "categoryCode=tennis&"  # Specify the desired category
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        sports_set = set()
        competitions_set = set()

        for event in data.get('data', {}).get('events', []):
            # Collect match data
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Collect sports and competitions
            sports_set.add(match["sport"])
            competitions_set.add(match["competition"])

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)

            matches.append(match)

        # Add matches to the all_matches list
        all_matches.extend(matches)

        # Add sports and competitions to the range summary
        range_summary[f"{start}-{start + 200}"] = {
            "sports": list(sports_set),
            "competitions": list(competitions_set)
        }

        # Sleep for a random duration between 1 and 2 seconds
        time.sleep(random.uniform(1, 2.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the range summary
for range_key, summary in range_summary.items():
    print(f"\nDrillDownTagIds Range: {range_key}")
    print(f"Sports: {summary['sports']}")
    print(f"Competitions: {summary['competitions']}")

# Save the match data to a CSV or Excel file if needed
final_matches_df.to_csv("matches_output.csv", index=False)

# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv("range_summary.csv")

An error occurred for drilldown ID range 0–200: 'NoneType' object is not iterable

DrillDownTagIds Range: 200-400
Sports: []
Competitions: []

DrillDownTagIds Range: 400-600
Sports: ['Voetbal', 'Rugby Union', 'Darten', 'Volleybal', 'Snooker']
Competitions: ['Israel Liga Leumit', 'Servië Wiener Städtische Liga', 'Roemenië Liga 1', 'Portugal Segunda Liga', 'Oostenrijk Volley League', 'Portugal Divisao Mannen', 'United Rugby Championship', 'Frankrijk Pro D2', 'Zweden Elitserien Mannen', 'Superliga Femenina', 'Modus Darts Online Live League', 'Zweden Eliteserien Vrouwen', 'Griekenland Super League', 'Duitsland Bundesliga', 'Spanje LaLiga', 'Portugal Primeira Liga', 'Griekenland Volleyleague', 'Engeland League 1', 'Polen PlusLiga', 'Engeland Premier League', 'Italy Serie A2', 'Duitsland DVL 1. Bundesliga', 'Polen 1. Liga', 'Engeland Premiership', 'Frankrijk Top 14\n', 'Israel Premier League', 'Italië Serie A1, Vrouwen', 'Brazilian Superliga B', 'Duitsland 3.Liga', 'Turkije 1. Lig', 'Bulgari

In [16]:
final_matches_df[(final_matches_df['sport'] == 'Voetbal') & (final_matches_df['competition'].str.contains('Champions League'))]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
983,6248696,PSV vs Liverpool,2025-01-29T20:00:00Z,PSV,Liverpool,UEFA Champions League,Europees,Voetbal,2.80,2.37
984,6248695,VfB Stuttgart vs Paris Saint Germain FC,2025-01-29T20:00:00Z,VfB Stuttgart,Paris Saint Germain FC,UEFA Champions League,Europees,Voetbal,3.55,2.70
985,6248651,Barcelona vs Atalanta,2025-01-29T20:00:00Z,Barcelona,Atalanta,UEFA Champions League,Europees,Voetbal,1.75,4.25
986,6248684,Stade Brest vs Real Madrid,2025-01-29T20:00:00Z,Stade Brest,Real Madrid,UEFA Champions League,Europees,Voetbal,7.00,1.41
987,6248647,Aston Villa FC vs Celtic,2025-01-29T20:00:00Z,Aston Villa FC,Celtic,UEFA Champions League,Europees,Voetbal,1.54,5.75
988,6248658,Bayer Leverkusen vs AC Sparta Praag,2025-01-29T20:00:00Z,Bayer Leverkusen,AC Sparta Praag,UEFA Champions League,Europees,Voetbal,1.14,17.50
989,6248665,Manchester City vs Club Brugge,2025-01-29T20:00:00Z,Manchester City,Club Brugge,UEFA Champions League,Europees,Voetbal,1.19,13.50
990,6248683,SK Sturm Graz vs RB Leipzig,2025-01-29T20:00:00Z,SK Sturm Graz,RB Leipzig,UEFA Champions League,Europees,Voetbal,3.30,2.06
991,6248649,Borussia Dortmund vs FC Shakhtar Donetsk,2025-01-29T20:00:00Z,Borussia Dortmund,FC Shakhtar Donetsk,UEFA Champions League,Europees,Voetbal,1.38,7.25
992,6248654,Dinamo Zagreb vs AC Milan,2025-01-29T20:00:00Z,Dinamo Zagreb,AC Milan,UEFA Champions League,Europees,Voetbal,5.50,1.53


In [17]:
# range starts at 1500 and stops at 13000

In [18]:
# range_summary

In [19]:
# keys_with_basketbal = [key for key, value in range_summary.items() if 'Voetbal' in value.get('sports', [])]
# keys_with_basketbal

In [20]:
# keys_with_voetbal = [key for key, value in range_summary.items() if 'Basketbal' in value.get('sports', [])]
# keys_with_voetbal